In [1]:
import re
import numpy as np
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from bs4 import BeautifulSoup
import re
import os
import codecs
from sklearn import feature_extraction
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from bokeh.io import push_notebook, output_notebook
from bokeh.models import ColumnDataSource, LabelSet,HoverTool
from bokeh.plotting import figure, output_file, show
from bokeh.models import (ColumnDataSource,HoverTool,LogColorMapper,BasicTicker,PrintfTickFormatter,ColorBar)
lines = []
with open("Paraiso Edits.txt") as file:
    for line in file: 
        line = line.strip('\n')
        lines.append(line)

D:\Anaconda\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
words_of_interest = ['Undid','Reverted','Reverting']
lines_of_interest = []

for line in lines:
    for word in words_of_interest:
        if word in line:
            lines_of_interest.append(line)

In [3]:
ops = []

for line in lines_of_interest:
    if '2007' in line:
        poster = line[line.rindex('2007')+5:].split()[0]
    elif '2006' in line:
        poster = line[line.rindex('2006')+5:].split()[0]
    else:
        pass
    ops.append(poster)

In [4]:
undone = []

for line in lines_of_interest:
    if 'Undid' in line:
        corrected = line[line.rindex('by')+2:].split()[0].replace('.','').replace(')','').replace(';','').replace('"','')
    elif 'Reverted' in line:
        corrected = line[line.rindex('by')+2:].split()[0].replace('.','').replace(')','').replace(';','').replace('"','')
    elif 'Reverting' in line:
        corrected = line[line.rindex('by')+2:].split()[0].replace('.','').replace(')','').replace(';','').replace('"','')
    
    undone.append(corrected)

In [5]:
l = lines_of_interest[4]
l

'# (cur) (last) 16:24, 10 January 2007 Edemir (Talk | contribs) m (102,458 bytes) (Reverted good faith edits by Dragon; No need to dampen it; the reference points to a blanket opinion of psychiatry and psychology.. using TW)'

In [6]:
l[l.rindex('by')+2:].split()[0]

'Dragon;'

In [7]:
weights = []

for line in lines_of_interest:
    weights.append(len(line[line.rindex('bytes')+7:]))

In [8]:
len(weights)

220

In [9]:
# __authors__ = "Ajay Shaan Shanmugam;Siddharth Chandrasekaran"
__license__ = "GPL"
__version__ = "1.0.1"
__emails__ = "ashanmugam@umass.edu;schandraseka@umass.edu"

import pandas as pd

from random import randint

import networkx as nx

from collections import namedtuple

from math import sqrt

import bokeh

from bokeh.models import HoverTool

from bokeh.plotting import show, figure

from bokeh.colors import RGB

import random

import community.community_louvain as community

from bokeh.models import ColumnDataSource,HoverTool,ColorBar,LabelSet

#data = pd.read_csv('CellPhoneCallRecords.csv')



#weights = data['Duration(seconds)'].tolist()


callers = ops
receivers = undone


edgelist = pd.DataFrame({'callers':callers,'receivers':receivers,'weights':weights})
#Normalizing for displaying
edgelist['weights'] = edgelist['weights']/100

all_nodes = list(set(callers))

for rec in list(set(receivers)):
    if rec in all_nodes:
        continue
    else:
        all_nodes.append(rec)
        


def generate_coordinates(m, n):
    seen = set()

    x, y = randint(m, n), randint(m, n)

    while True:
        seen.add((x, y))
        yield [x, y]
        x, y = randint(m, n), randint(m, n)
        while (x, y) in seen:
            x, y = randint(m, n), randint(m, n)

g = generate_coordinates(1,400)
xs = []
ys = []

for i in range(len(all_nodes)):
    xy = next(g)
    x = xy[0]
    y = xy[1]
    xs.append(x)
    ys.append(y)
nodelist = pd.DataFrame({'nodes':all_nodes,'x':xs,'y':ys})


def create_bokeh_graph(graph, clus_algo, title = None):

    

    def gen_edge_coordinates(graph, layout):

        xs = []

        ys = []

        val = namedtuple("edges", "xs ys")

        for edge in graph.edges():

            from_node = layout[edge[0]]

            to_node = layout[edge[1]]

            xs.append([from_node[0],to_node[0]])

            ys.append([from_node[1], to_node[1]])

        return val(xs=xs, ys=ys)



    def gen_node_coordinates(layout):

        names, coords = zip(*layout.items())

        xs, ys = zip(*coords)

        val = namedtuple("nodes", "names xs ys")

        return val(names=names, xs=xs, ys=ys)



    plot_layout = nx.circular_layout(graph)

    edges = graph.edges()
    
    weights = [graph[u][v]['weight'] for u,v in edges]

    nx.draw(graph, plot_layout, edges=edges, width=weights)

    _nodes = gen_node_coordinates(plot_layout)

    _edges = gen_edge_coordinates(graph, plot_layout)


    hover = HoverTool(tooltips=[('name', '@name'), 

                                ('node_id', '$index'),

                                ('degree', '@degree'),

                                ('cluster_no', '@community_nr')], names=["show_hover"])



    plot = figure(width=700, height=700, 

                 tools=[hover, 'box_zoom', 'resize', 'reset', 'wheel_zoom', 'pan', 'lasso_select'],

                logo = None)
    
    if title == None:
    
        plot.title.text ="Algorithm : " + clus_algo
    else:
        plot.title.text = title

    plot.toolbar.logo = None

    plot.axis.visible = False                            

    plot.xgrid.grid_line_color = None

    plot.ygrid.grid_line_color = None

    

    #Draw Edges

    source_edges = bokeh.models.ColumnDataSource(dict(xs=_edges.xs, ys=_edges.ys))

    plot.multi_line('xs', 'ys', line_color='navy', source=source_edges, alpha=0.17, line_width=weights)

    

    #best partition based on the algorithm selected

    degrees = list(nx.degree(graph).values())
    
    if clus_algo == "triangle":
        
        clustering = list(nx.triangles(graph).values())
    
    elif clus_algo == "square":
        
        clustering = list(nx.square_clustering(graph).values())
    
    else:
        
        clustering = list(nx.clustering(graph).values())
        
    best_partition = community.best_partition(graph)

    nodes, communities = zip(*best_partition.items())

    betw = list(nx.betweenness_centrality(graph).values())

    

    #Color mapper

    colormapper = {x : RGB(random.randrange(0,256),random.randrange(0,256),random.randrange(0,256)) 

                     for x in list(set(communities))}

    cluster_color_list, community_nr = zip(*[(colormapper[best_partition[node]], best_partition[node]) for node in nodes])



    

    graph_nodes = graph.number_of_nodes()

    

    colors = ['firebrick' for node in range(graph_nodes)]

    

    #Drawing circles for nodes

    source_nodes = bokeh.models.ColumnDataSource(dict(xs=_nodes.xs, ys=_nodes.ys, name=_nodes.names, 

                                                      single_color = colors,

                                                      color_by_cluster = cluster_color_list, 

                                                      degree=degrees, 

                                                      clustering=clustering, community_nr=community_nr,

                                                      betweenness = betw))

    

    r_circles = plot.circle('xs', 'ys', fill_color='single_color', line_color='single_color', 

                          source = source_nodes, alpha=0.7, size=9, name="show_hover")
    
    #plot.circle(0.538678941825779,0.586073848441823,fill_color='#000000',size=12)
    
 

    colorcallback = bokeh.models.callbacks.CustomJS(args=dict(source=source_nodes, circles=r_circles), code="""

        var value = cb_obj.get('value');

        circles.glyph.line_color.field = value;

        circles.glyph.fill_color.field = value;

        source.trigger('change')

    """)  

    

    button = bokeh.models.widgets.Select(title="Color", value="single_color", 

                                         options=["single_color", "color_by_cluster"], 

                                         callback=colorcallback)
    
    
    
    return [plot,button]
    




g = nx.Graph()

for i, elrow in edgelist.iterrows():
    g.add_edge(elrow['callers'], elrow['receivers'], weight=elrow['weights'])
    
for i, nlrow in nodelist.iterrows():
    g.node[nlrow['nodes']] = nlrow[1:].to_dict()
    
#data = pd.read_csv('CellPhoneCallRecords_New.csv')
#data = data[(data['From']==200) | (data['To']==200)]

#weights = (data['Duration(seconds)'] * data['Frequency']).tolist()


callers = ops
receivers = undone


edgelist1 = pd.DataFrame({'callers':callers,'receivers':receivers,'weights':weights})
#Normalizing for displaying
edgelist1['weights'] = edgelist1['weights']/100

all_nodes = list(set(callers))

for rec in list(set(receivers)):
    if rec in all_nodes:
        continue
    else:
        all_nodes.append(rec)
        


def generate_coordinates(m, n):
    seen = set()

    x, y = randint(m, n), randint(m, n)

    while True:
        seen.add((x, y))
        yield [x, y]
        x, y = randint(m, n), randint(m, n)
        while (x, y) in seen:
            x, y = randint(m, n), randint(m, n)

g3 = generate_coordinates(1,400)
xs = []
ys = []

for i in range(len(all_nodes)):
    xy = next(g3)
    x = xy[0]
    y = xy[1]
    xs.append(x)
    ys.append(y)
nodelist1 = pd.DataFrame({'nodes':all_nodes,'x':xs,'y':ys})

g1 = nx.Graph()

for i, elrow in edgelist1.iterrows():
    g1.add_edge(elrow['callers'], elrow['receivers'], weight=elrow['weights'])
    
for i, nlrow in nodelist1.iterrows():
    g1.node[nlrow['nodes']] = nlrow[1:].to_dict()

layout_plot = []
layout_plot.append(create_bokeh_graph(g, "triangle"))
#layout_plot.append(create_bokeh_graph(g, "square"))
#layout_plot.append(create_bokeh_graph(g, "clustering"))
layout_plot1 = []
#layout_plot1.append(create_bokeh_graph(g1, "clustering", "Visualization of Ferdinando's connections"))
    #Create grid and save

#data = pd.read_csv('CellPhoneCallRecords.csv')
#data = data[(data['From']==137)| (data['To']==137)| (data['From']==2)| (data['To']==2)| (data['From']==3)| (data['To']==3)]

#weights = (data['Duration(seconds)'] ).tolist()


callers = ops
receivers = undone


edgelist1 = pd.DataFrame({'callers':callers,'receivers':receivers,'weights':weights})
#Normalizing for displaying
edgelist1['weights'] = edgelist1['weights']/100

all_nodes = list(set(callers))

for rec in list(set(receivers)):
    if rec in all_nodes:
        continue
    else:
        all_nodes.append(rec)
        


def generate_coordinates(m, n):
    seen = set()

    x, y = randint(m, n), randint(m, n)

    while True:
        seen.add((x, y))
        yield [x, y]
        x, y = randint(m, n), randint(m, n)
        while (x, y) in seen:
            x, y = randint(m, n), randint(m, n)

g3 = generate_coordinates(1,400)
xs = []
ys = []

for i in range(len(all_nodes)):
    xy = next(g3)
    x = xy[0]
    y = xy[1]
    xs.append(x)
    ys.append(y)
nodelist1 = pd.DataFrame({'nodes':all_nodes,'x':xs,'y':ys})

g1 = nx.Graph()

for i, elrow in edgelist1.iterrows():
    g1.add_edge(elrow['callers'], elrow['receivers'], weight=elrow['weights'])
    
for i, nlrow in nodelist1.iterrows():
    g1.node[nlrow['nodes']] = nlrow[1:].to_dict()

#layout_plot1.append(create_bokeh_graph(g1, "clustering", "Visualization of David Vidro, Juan Vidro and Jorge Vidro's connections"))

layout_plot = bokeh.layouts.gridplot(layout_plot+layout_plot1)
show(layout_plot)



D:\Anaconda\lib\site-packages\networkx\drawing\nx_pylab.py:126: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
  b = plt.ishold()
D:\Anaconda\lib\site-packages\networkx\drawing\nx_pylab.py:138: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
  plt.hold(b)
D:\Anaconda\lib\site-packages\matplotlib\__init__.py:917: UserWarning: axes.hold is deprecated. Please remove it from your matplotlibrc and/or style files.
  warnings.warn(self.msg_depr_set % key)
D:\Anaconda\lib\site-packages\matplotlib\rcsetup.py:152: UserWarning: axes.hold is deprecated, will be removed in 3.0
  warnings.warn("axes.hold is deprecated, will be removed in 3.0")
D:\Anaconda\lib\site-packages\bokeh\util\depr

In [10]:

stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")


def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems



lines = []
with open("Paraiso-Edits.txt") as file:
    for line in file: 
        line = line.strip('\n')
        lines.append(line)
lines_of_interest = []
words_of_interest = ["Paraiso", "paraiso", "Catalano", "catalano"]
for line in lines:
    op = "Not_Applicable"
    flag = False
    for word in words_of_interest:
        if word in re.findall("\((.*?)\)", line)[-1]:
            lines_of_interest.append(re.findall("\((.*?)\)", line)[-1])
            flag = True
    if '2007' in line:
        op = line[line.rindex('2007')+5:].split()[0]
    elif '2006' in line:
        op = line[line.rindex('2006')+5:].split()[0]
    if op in line:
        lines_of_interest.append(re.findall("\((.*?)\)", line)[-1])
        ops.append(op)
    if flag:
        ops.append(op)

allwords_stemmed = []      
for line in lines_of_interest:
    line = re.sub('[!@#$]', '', line)
    allwords_stemmed.append(tokenize_and_stem(line))


In [11]:
from bokeh.layouts import column
model = Word2Vec(sentences=allwords_stemmed, # tokenized senteces, list of list of strings
                 size=300,  # size of embedding vectors
                 workers=4, # how many threads?
                 min_count=20, # minimum frequency per token, filtering rare words
                 sample=0.05, # weight of downsampling common words
                 sg = 0, # should we use skip-gram? if 0, then cbow
                 iter=5,
                 hs = 0
        )
print(len(model.wv.vocab.keys()))
#print(len(X_tsne))
X = model[model.wv.vocab]
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)

bokeh_df = pd.DataFrame(X_tsne,model.wv.vocab.keys(), columns=['x','y'])

bokeh_df['text_labels'] = bokeh_df.index

    # interactive controls to include to the plot
TOOLS="hover, zoom_in, zoom_out, box_zoom, undo, redo, reset, box_select"

p = figure(tools=TOOLS, plot_width=700, plot_height=700)

# define data source for the plot
source = ColumnDataSource(bokeh_df)

# scatter plot
t = p.scatter('x', 'y', source=source, fill_alpha=0.6,
              fill_color="#8724B5",
              line_color=None)

# text labels
labels = LabelSet(x='x', y='y', text='text_labels', y_offset=8,
                      text_font_size="6pt", text_color="#555555",
                      source=source, text_align='center')

p.add_layout(labels)

output_notebook()

show(p)


74


Loading BokehJS ...